In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('titanic_train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df['Sex'].unique()

array(['male', 'female'], dtype=object)

In [9]:
df.loc[df["Sex"] == "male", "Sex"] = 0
df.loc[df['Sex'] == 'female', 'Sex'] = 1

In [11]:
df['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [13]:
df["Embarked"] = df["Embarked"].fillna('S')
df.loc[df["Embarked"] == "S", "Embarked"] = 0
df.loc[df['Embarked'] == 'C', 'Embarked'] = 1
df.loc[df['Embarked'] == 'Q', 'Embarked'] = 2

In [14]:
df['Embarked'].unique()

array([0, 1, 2], dtype=object)

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold

predictors = ['Pclass','Sex', 'Age', 'SibSp', 'Parch','Fare','Embarked']

alg1 = LinearRegression()

kf = KFold(df.shape[0], n_folds=3, random_state=1)

predictions = []

for train, test in kf:
    # 获取训练数据
    train_predictiors = (df[predictors].iloc[train, :])
    train_target = (df['Survived'].iloc[train])
    
    # 训练模型
    alg1.fit(train_predictiors, train_target)
    
    # 对测试数据进行预测
    test_predictiors = alg1.predict(df[predictors].iloc[test, :])
    
    # 将本次结果添加到predictions
    predictions.append(test_predictiors)

In [32]:
import numpy as np

In [33]:
predictions = np.concatenate(predictions, axis=0)
predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0
accuracy = sum(predictions == df['Survived'])/len(predictions)
print(accuracy)

0.7833894500561167


In [34]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

alg2 = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(alg2, df[predictors], df['Survived'], cv=3)
print(scores)
print(scores.mean())

[0.78451178 0.78787879 0.79124579]
0.7878787878787877


/Users/pengyi/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/pengyi/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/pengyi/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
# 读取测试数据
test_df = pd.read_csv('titanic_train.csv')
test_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
# 对测试数据进行处理
test_df['Age'] = test_df['Age'].fillna(df['Age'].median())
test_df.loc[test_df["Sex"] == "male", "Sex"] = 0
test_df.loc[test_df['Sex'] == 'female', 'Sex'] = 1
test_df["Embarked"] = test_df["Embarked"].fillna('S')
test_df.loc[test_df["Embarked"] == "S", "Embarked"] = 0
test_df.loc[test_df['Embarked'] == 'C', 'Embarked'] = 1
test_df.loc[test_df['Embarked'] == 'Q', 'Embarked'] = 2

In [41]:
# 使用上面训练的模型预测,并查看准确率
test_predictiors = alg1.predict(test_df[predictors])
test_predictiors[test_predictiors > 0.5] = 1
test_predictiors[test_predictiors <= 0.5] = 0
accuracy = sum(test_predictiors == test_df['Survived'])/len(test_predictiors)
print(accuracy)

0.797979797979798
